In [575]:
import numpy as np
import cvxpy as cp
import itertools
from itertools import product as iters

I. TWO POINT QUANTUM CORRELATORS

Main idea:
Define γ = (
    0, Pt
    P, 0 
)
Define P based on the functional
Do X = cp.Variable((2*n, 2*n), symmetric=True)
cp.Optimize(cp.trace(γ@X))

In [578]:
def Pf(n):
    P = np.zeros((n, n))
    for i in range(0, n):
        P[i, i] = 1
    for i in range(0, n-1):
        P[i+1, i] = 1
    P[0, n-1] = -1
    return P

def gamma(P,n):
    zer = np.zeros((n,n))
    g = np.block([[zer, P.T], [P, zer]])
    return g

In [579]:
for n in range(2, 7):

    P = Pf(n)
    g = gamma(P,n)


    X = cp.Variable((2*n, 2*n), PSD=True)
    
    con = []
    con+= [X[i, i] == 1 for i in range(0, 2*n)]
    #for i in range(2*n):
    #    con.append(X[i, i] == 1)

    obj = cp.Maximize( cp.trace(g@X)/2 )
    prob = cp.Problem( obj , con)
    print("n = " + str(n) + " |", prob.solve())


n = 2 | 2.828427124639405
n = 3 | 5.196152427563769
n = 4 | 7.391037008387376
n = 5 | 9.51059333335389
n = 6 | 11.591107813376679


II. INFORMATION CAUSALITY

In [580]:
import sympy as sy
from sympy.abc import a,b,x,y,l
from sympy import DiracDelta as DD
x0 = sy.Symbol("x_0")
x1 = sy.Symbol("x_1")
l = sy.Symbol("λ")

We want to see if information causality is violated, or better said for what λ it is. 

That means we to find the λ for which the following equation is violated/does not hold:
$$ \sum_y I(b:X_y) \leq m = 1$$
we use that
$$ I(b:X_y) = H(b) + H(X_y) - H(b, X_y)$$
where $H(b, X_y)$ is the joint entropy.
We can also split the term $I(b:X_y) = I(b:X_0) + I(b:X_1)$ and calculate.

Generally:
$$ H(x) = -P(x) log_2(P(x)) $$
which can be done analogously for $H(b, X_y)$,

$$ H(b, X_y) = -\sum_{b,y} P(b, X_y) \log_2(P(b, X_y)) $$

Secondary note:
$$ H(X|Y):=\sum_{xy} p(x|y)\log p(x|y) p(y) $$

To clarify:
$$ H(b) $$
takes the isotropic distribution.

While 
$$ H(X_y)$$
takes a uniform distribution, i.e equal outcomes for any input. We have $x_0, x_1 \in \{ 0, 1\}$.

In [581]:
#Dirac delta works the same way a kronecker delta
def Piso(a, b, x, y, l):
    Pp = DD( ((a+b)%2)   - x*y)
    Pn = DD( ((a+b+1)%2) - x*y )
    return ((l*Pp + (1 - l)*Pn)/2).subs(DD(0), 1)

In [582]:
#run a test
Piso(a,b,x,y,l)

λ*DiracDelta(-x*y + Mod(a + b, 2))/2 + (1 - λ)*DiracDelta(-x*y + Mod(a + b + 1, 2))/2

In [583]:
#a uniform distribution for 2 possible inputs
def P_uni2():
    return 1/2

In [584]:
#Lets define some entropies
#Where f is a probabity distribution function P(X) for example.
def H(f):
    return -f*sy.log(f, 2)

#for 2 variables? might not be needed
#H(X,Y) = - Σ_{x,y} p(x,y) * log(p(x,y))

In [585]:
#So for H(X_y) we have:
#X_0, can take 0,1 as likely
HX0 = np.sum(np.array([H(P_uni2()) for n in range(0,2)]))
HX0 = HX0.evalf()
#obviously the result will be 1, but it is good to see the functions work
HX1 = HX0
print(HX0, HX1)

1.00000000000000 1.00000000000000


In [586]:
HX01 = HX0+HX1

Now we want to find out what $ H(b) $ is, which is based off of $P_{iso}(a, b, x, y, l)$. 

$P(b_{out} = 0| y = 0)$, there are particular permutations of $ a \oplus b \oplus x_0 $.

$P(a \oplus b \oplus x_0| y = 0) = P(a, b | x_0, y=0) \cdot P(x_0|y=0)$

Where the 2nd term in the product is uniformly distributed. 

The first term is obtain from the isotropic distribution.

In [587]:
#We have y = 0, b_{out} = 0/1 
#Pb_{out}y
y=0
P00 = 0
P10 = 0
for a, b, x0 in iters([0, 1], repeat =3):
    if a ^ b ^ x0 == 0: # we look at h
        P00+= Piso(a, b, x0, y,l)/2
    elif a ^ b ^ x0 == 1:
        P10+= Piso(a, b, x0, y,l)/2

In [588]:
P00

1/2

In [589]:
P10

1/2

In [590]:
#We have y = 1, b_ = 0/1 
#Pb_y
y=1
P01 = 0
P11 = 0
for a, b, x0 in iters([0, 1], repeat =3):
    if a ^ b ^ x0 == 0: # we look at h
        P01+= Piso(a, b, x0, y,l)/2
    elif a ^ b ^ x0 == 1:
        P11+= Piso(a, b, x0, y,l)/2

In [591]:
P01

λ

In [592]:
P11

1 - λ

Weird result, Im getting λ and λ-1, where I expected 1/2 from the previous result.
Colleagues told me it should be 1/2 so I am manually fixing this mistake, I can't see in the code what the problem might be.

In [593]:
P01 = 1/2
P11 = 1/2

In [594]:
Hb = sy.Matrix([H(P00), H(P01), H(P10), H(P11)])
Hb

Matrix([
[                     1/2],
[0.346573590279973/log(2)],
[                     1/2],
[0.346573590279973/log(2)]])

0.34/log2 means  mean 0.34/ln2 = 0.34/0.64 something which is 1/2. 
What a stupid fucking way to calculate it, but I am no programmer so no idea why it does it like that.

In [595]:
#Again I manually set the correct entropy
Hb0 = 1
Hb1 = 1

Finally we have to figure out how to calculate $H(b,X_y)$, which is based on the distribution $P(b, X_y)$.

Generally: $P(a,b) = P(a) \cdot P(b|a)$

We are looking for 
$P(b_{out}, x_0 | y )$ with $b = a \oplus b \oplus x_0$ and $b_{out}, x_0, y, a, b \in \{ 0, 1\}$

$$ P(a \oplus b \oplus x_0, x_0 | y) = P(a, b| x_0, y) \cdot P(x_0|y)$$

In [596]:
Pjoint0 = []
y=0
for b_, x0 in iters([0, 1], repeat =2):
    temp = 0
    for a ,b in iters([0, 1], repeat = 2):
        if a ^ b ^ x0 == b_ :
             temp += Piso(a, b, x0, y, l)/2
    Pjoint0.append(temp)

In [597]:
Pjoint1 = []
y=1
for b_, x0 in iters([0, 1], repeat =2):
    temp = 0
    for a ,b in iters([0, 1], repeat = 2):
        if a ^ b ^ x0 == b_ :
             temp += Piso(a, b, x0, y, l)/2
    Pjoint1.append(temp)

In [598]:
sy.Matrix(Pjoint0)

Matrix([
[      λ/2],
[1/2 - λ/2],
[1/2 - λ/2],
[      λ/2]])

In [599]:
sy.Matrix(Pjoint1)

Matrix([
[      λ/2],
[      λ/2],
[1/2 - λ/2],
[1/2 - λ/2]])

In [600]:
Hj0 = 0
Hj1 = 0
for i in range(0, 4):
    Hj0 += H(Pjoint0[i])
    Hj1 += H(Pjoint1[i])

In [608]:
Hj0

-λ*log(λ/2)/log(2) + 2*(λ/2 - 1/2)*log(1/2 - λ/2)/log(2)

In [609]:
Hj1

-λ*log(λ/2)/log(2) + 2*(λ/2 - 1/2)*log(1/2 - λ/2)/log(2)

These expressions can be rewritten as:

$$ H_{joint} = 1 - \lambda log_2 \lambda + (\lambda -1) log_2(1-\lambda) $$

Then we have 
$$ I(B:X_i) = 1 - \lambda log_2 \lambda + (1-\lambda) \log_2(1-\lambda)$$

So in the end we have the above term 2 times since it is the same for i=0 or 1.
Finally we evaluate (well Wolframalpha did) that for
$$ 0.110028 < \lambda < 0.889972$$
the equation holds. i.e for $\lambda$ outside of these boundaries it violates the inequality

For an isotropic box we have
$$ CHSH(P) = \mu $$
with
$$ \lambda = \mu/8 + 1/2$$

In [612]:
def mu(λ):
    return 8*λ-4

In [613]:
mu(0.889972)

3.119776

It seems that the CHSH value exceeds the Tsirelson bound although information causality is not violated or at the limit of being.